In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit

In [2]:
spark = SparkSession.builder.appName("bk-imp").getOrCreate()

23/04/17 15:53:54 WARN Utils: Your hostname, workspace resolves to a loopback address: 127.0.1.1; using 11.11.1.73 instead (on interface eth0)
23/04/17 15:53:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/17 15:53:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("../data/meta_Digital_Music.json").select(
    ["also_buy", "also_view", "asin"]
)

In [4]:
df.printSchema()

root
 |-- also_buy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- also_view: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- asin: string (nullable = true)



In [5]:
# Explode also_buy
also_buy_df = df.select(
    col("asin").alias("src_product_id"),
    explode("also_buy").alias("dst_product_id"),
    lit("also_buy").alias("relationship"),
)

# Explode also_view
also_view_df = df.select(
    col("asin").alias("src_product_id"),
    explode("also_view").alias("dst_product_id"),
    lit("also_buy").alias("relationship"),
)

# Union the two dataframes
result_df = also_buy_df.union(also_view_df).dropDuplicates(
    ["src_product_id", "dst_product_id"]
)

In [6]:
result_df.show(10)

+--------------+--------------+------------+
|src_product_id|dst_product_id|relationship|
+--------------+--------------+------------+
|    0001377647|    0345538374|    also_buy|
|    0001377647|    0439763096|    also_buy|
|    0001377647|    B000008LDC|    also_buy|
|    0001377647|    B000008LDG|    also_buy|
|    0001377647|    B000008LDL|    also_buy|
|    0001377647|    B000008LDP|    also_buy|
|    0001377647|    B000008N3O|    also_buy|
|    0001377647|    B00004UU0Y|    also_buy|
|    0001377647|    B000088E97|    also_buy|
|    0001377647|    B0000AM6KG|    also_buy|
+--------------+--------------+------------+
only showing top 10 rows

